In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):
    def  __init__(self, img_dim) -> None:
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
    def forward(self,x):
        return self.disc(x)

In [3]:
class Generator(nn.Module):
    def  __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim), # MNIMST 28*28*1
            nn.Tanh(),
        )
    def forward(self,x):
        return self.gen(x)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-5
z_dim = 64
image_dim = 28*28*1
batch_size = 64
num_epochs = 50

In [5]:
discriminator = Discriminator(image_dim).to(device)
generator = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))] # means and std of MNIST 
)
dataset = datasets.MNIST(root = "dataset/", transform=transforms, download = True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(discriminator.parameters(), lr=lr)
opt_gen = optim.Adam(generator.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [8]:
print(loader)

In [14]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1,784).to(device)
        batch_size = real.shape[0]
        ### Train Discriminator: max (log(D(real)) + log(1 - D(fake)))
        noise = torch.randn(batch_size, z_dim).to(device)  # random noise
        fake = generator(noise)                          # generate a fake image
        disc_real = discriminator(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = discriminator(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2 
        discriminator.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Discriminator: min (log(1 - D(G(z)) --> max log(D(G(z))))
        output = discriminator(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        generator.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        ## tensorboard
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = generator(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch [0/50] Batch 0/938                       Loss D: 0.6777, loss G: 0.7149
Epoch [1/50] Batch 0/938                       Loss D: 0.5810, loss G: 0.5340
Epoch [2/50] Batch 0/938                       Loss D: 0.5731, loss G: 0.5959
Epoch [3/50] Batch 0/938                       Loss D: 0.4914, loss G: 0.7129
Epoch [4/50] Batch 0/938                       Loss D: 0.4439, loss G: 0.7478
Epoch [5/50] Batch 0/938                       Loss D: 0.4382, loss G: 0.7174
Epoch [6/50] Batch 0/938                       Loss D: 0.4441, loss G: 0.6634
Epoch [7/50] Batch 0/938                       Loss D: 0.4113, loss G: 0.7378
Epoch [8/50] Batch 0/938                       Loss D: 0.3947, loss G: 0.8238
Epoch [9/50] Batch 0/938                       Loss D: 0.3806, loss G: 0.8613
Epoch [10/50] Batch 0/938                       Loss D: 0.3734, loss G: 0.8804
Epoch [11/50] Batch 0/938                       Loss D: 0.3594, loss G: 0.9177
Epoch [12/50] Batch 0/938                       Loss D: 0.3370